In [94]:
import requests
import pandas as pd
import json
import re
from bs4 import BeautifulSoup
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
import math

# Extraer información de los json
Estructura del json:
```json
{
    "links": [],
    "metadata": {},
    "items": [
        {
            ...
            "title": "titulo del articulo",
            "abstractText": ["abstract del articulo"]
        }
    ]
}
```


## Nombres de los json:

### editorial.json
Contiene: --- <br>
Necesitamos: 153

### book_chapter.json
Contiene: 200 <br>
Necesitamos: 122

### review.json
Contiene: 200 <br>
Necesitamos 99



### Estrategia de exacción
Sacar la cantidad de items necesarios, sacando el nombre y el abstract para colocarlo en un dataset de pandas.

In [37]:
# Extracción de datos de reviews
def extract_info_from_json(file_path, target_count):
    with open(file_path, 'r') as file:
        data = json.load(file)
    # Filtrar elementos que tienen 'abstractText' no vacío
    filtered_items = [item for item in data['items'] if 'abstractText' in item and item['abstractText']]
    # Limitar a target_count si hay suficientes elementos
    limited_items = filtered_items[:target_count]
    extracted_info = [
        {
            'title': item['title'],
            'abstractText': ' '.join(item['abstractText']),
            'source': file_path  # Añadir el nombre del archivo como parte de la información extraída
        }
        for item in limited_items
    ]
    return extracted_info

In [36]:
# Definir los archivos y la cantidad de items a extraer
files_and_items = {
    'editorial.json': 153,
    'book_chapter.json': 122,
    'review.json': 99
}

In [50]:
# Extraer la información y convertirla en DataFrames
dataframes = []
for file_path, num_items in files_and_items.items():
    info = extract_info_from_json(file_path, num_items)
    df = pd.DataFrame(info)
    dataframes.append(df)

In [51]:
# Concatenar todos los DataFrames en uno solo
final_df = pd.concat(dataframes, ignore_index=True)

In [71]:
# Descargar la lista de stop words
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladimircuriel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
# Obtener las stop words en inglés
stop_words = set(stopwords.words('english'))

In [74]:
# Función para limpiar el texto
def clean_text(text):
    # Eliminar etiquetas HTML
    text = re.sub(r'<[^>]+>', '', text)
    # Convertir a minúsculas
    text = text.lower()
    # Quitar signos de puntuación
    text = re.sub(r'[^\w\s]', '', text)
    # Dividir el texto en palabras
    words = word_tokenize(text)
    # Filtrar las stop words
    filtered_words = [word for word in words if word not in stop_words]
    # Unir las palabras filtradas de nuevo en una cadena
    text = ' '.join(filtered_words)
    return text

In [95]:
# Aplicar la función de limpieza a las columnas deseadas
final_df['abstractText'] = final_df['abstractText'].apply(clean_text)
final_df['title'] = final_df['title'].apply(clean_text)

In [96]:
final_df

,title,abstractText,source,word_count
0,augmenting research methods foundation models ...,deep learning dl research made remarkable prog...,editorial.json,127
1,special issue soco 2022 new trends soft comput...,eight papers included special issue represent ...,editorial.json,72
2,computational models cognition humanautomated ...,discuss stateoftheart future directions develo...,editorial.json,118
3,special issue soco 2021 new trends soft comput...,seven papers included special issue represent ...,editorial.json,72
4,biomedical informatics state art challenges op...,biomedical informatics considered multidiscipl...,editorial.json,86
...,...,...,...,...
346,bioinspired algorithms feature engineering ana...,purpose natures evolution shaped intelligent b...,review.json,165
347,computer computer science computational thinki...,digital computers invented 1940s sophisticated...,review.json,113
348,navigating metaverse technical review emerging...,metaverse burgeoning virtual reality realm gar...,review.json,162
349,human activity prediction studies using wearab...,nowadays human activity recognition har system...,review.json,105


In [97]:
# Mostrar el DataFrame final
final_df.to_csv('output.csv', index=False)

## Obteniendo top 10 palabras comunes

In [79]:
# Combinar el texto de las columnas 'title' y 'abstractText' en una sola serie de texto
all_text = final_df['abstractText']

In [80]:
# Dividir el texto en palabras y aplanar la lista
words = word_tokenize(' '.join(all_text))

In [81]:
# Contar la frecuencia de cada palabra
word_counts = Counter(words)

In [82]:
# Encontrar las 10 palabras más comunes
most_common_words = word_counts.most_common(10)

In [83]:
# Imprimir las 10 palabras más comunes y sus frecuencias
for word, count in most_common_words:
    print(f"{word}: {count}")

science: 415
data: 342
learning: 332
research: 314
computer: 303
ai: 227
systems: 167
applications: 145
intelligence: 143
field: 143


## Obteniendo el promedio de la cantidad de palabras de los abstracts

In [84]:
#Calcular la cantidad de palabras en cada abstract
final_df['word_count'] = final_df['abstractText'].apply(lambda x: len(word_tokenize(x)))

In [86]:
# Calcular el promedio de estas cantidades
average_word_count = final_df['word_count'].mean()

In [93]:
print(f"El promedio de palabras por abstract es: {math.ceil(average_word_count)}")

El promedio de palabras por abstract es: 105


In [98]:
# Mostrar el DataFrame final
final_df.to_csv('output.csv', index=False)